Finding Similar Anime By Genre

we're trying to make an anime finder by genre using simple feature and jaccard similarity score. 

In [9]:
import numpy as np
import pandas as pd
import itertools
import collections
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import jaccard_similarity_score

Preprocessing

We split the genre in preprocessing, so later we can turn them to features.

In [10]:
animes = pd.read_csv("anime1.csv") 

animes['genre'] = animes['genre'].fillna('None') # filling 'empty' data
animes['genre'] = animes['genre'].apply(lambda x: x.split(', ')) # split genre into list of individual genre

genre_data = itertools.chain(*animes['genre'].values.tolist()) # flatten the list
genre_counter = collections.Counter(genre_data)
genres = pd.DataFrame.from_dict(genre_counter, orient='index').reset_index().rename(columns={'index':'genre', 0:'count'})
genres.sort_values('count', ascending=False, inplace=True)

print(genres)

            genre  count
9          Comedy   4645
8          Action   2845
13      Adventure   2348
14        Fantasy   2309
4          Sci-Fi   2070
19          Drama   2016
6         Shounen   1712
0            Kids   1609
18        Romance   1464
1   Slice of Life   1220
31         School   1220
21         Hentai   1141
23   Supernatural   1037
12          Mecha    944
17          Music    860
15     Historical    806
16          Magic    778
10          Ecchi    637
3          Shoujo    603
5          Seinen    547
7          Sports    543
25        Mystery    495
22    Super Power    465
33       Military    426
11         Parody    408
30          Space    381
2          Horror    369
24          Harem    317
36         Demons    294
27   Martial Arts    265
38       Dementia    240
29  Psychological    229
26         Police    197
35           Game    181
20        Samurai    148
40        Vampire    102
34       Thriller     87
37           Cars     72
28     Shounen Ai     65


Feature Extraction
The feature extraction is simple, a binary encoded vector of genre.

this shows which feature/genre each show has in binary (by id)

In [11]:
genre_map = {genre: idx for idx, genre in enumerate(genre_counter.keys())}
def extract_feature(genre):
    feature = np.zeros(len(genre_map.keys()), dtype=int)
    feature[[genre_map[idx] for idx in genre]] += 1
    return feature
    
anime_feature = pd.concat([animes['name'], animes['genre']], axis=1)
anime_feature['genre'] = anime_feature['genre'].apply(lambda x: extract_feature(x))
print(anime_feature.head(30))

                                                 name  \
0                                         Doron Coron   
1   Hengen Taima Yakou Karura Mau! Sendai Kokeshi ...   
2                                      Asylum Session   
3     Gekitou! Crush Gear Turbo: Kaizabaan no Chousen   
4   Monkey Punch: Manga Katsudou Daishashin - Crim...   
5             Buddy Complex: Daremo Shiranai Ashita e   
6                                         Power Stone   
7   Ore no Imouto ga Konnani Kawaii Wake ga Nai Sp...   
8                     Yojouhan Shinwa Taikei Specials   
9                                       White Fantasy   
10                             Mahou no Yousei Persia   
11                                    Kenka no Ato wa   
12                       Daisetsusan no Yuusha Kibaou   
13                                   Neko mo, Onda-ke   
14                                         Akiba-chan   
15                            Shounen Tokugawa Ieyasu   
16                    Hello Kit

Testing

In [12]:
test_data = anime_feature.take([0, 19, 1, 2, 16, 23, 6, 49, 220, 66])
for row in test_data.iterrows():
    print('Similar anime like {}:'.format(row[1]['name']))
    search = anime_feature.drop([row[0]]) # drop current anime
    search['result'] = search['genre'].apply(lambda x: jaccard_similarity_score(row[1]['genre'], x))
    search_result = search.sort_values('result', ascending=False)['name'].head(10)
    for res in search_result.values:
        print('\t{}'.format(res))
    print()

Similar anime like Doron Coron:
	Ocha-ken: Ryokutto Monogatari
	Peng You Town
	Nisshin Seifun Group CM
	Noobow
	Histoire d&#039;Etty
	Fuku-chan
	Noobow: Na Kokoro
	Miracle! Mimika
	Futari wa Nakayoshi: Goo to Sue
	Sakura Capusule

Similar anime like Samurai Noodles: The Originator:
	Ishinomori Shoutarou no Rekishi Adventure
	Hairy Tale
	Shaka no Shougai
	Watashitachi no Kurashi to Doboku
	Tsuyu no Hito Shizuku
	Chikara to Onna no Yo no Naka
	Noroma na Jiji
	Manga Nihon Emaki
	Yume Kakeru Kougen: Kiyosato no Chichi Paul Rusch
	Manga Nippon Mukashibanashi

Similar anime like Hengen Taima Yakou Karura Mau! Sendai Kokeshi Enka:
	Bishoujo Senshi Sailor Moon R: Make Up! Sailor Senshi
	Umezu Kazuo no Noroi
	Makura no Danshi
	Inuki Kanako Zekkyou Collection: Gakkou ga Kowai!
	C.L.A.Y.
	Gendai Kibunroku Kaii Monogatari
	Love Bites
	Ao Oni The Animation (Movie)
	Youkai Ningen Bem
	H. P. Lovecraft&#039;s The Dunwich Horror and Other Stories

Similar anime like Asylum Session:
	Dimension W: W no T

In [ ]:
for row in anime_feature.iterrows():
    print('Similar anime like {}:'.format(row[1]['name']))
    search = anime_feature.drop([row[0]])
    search['result'] = search['genre'].apply(lambda x: jaccard_similarity_score(row[1]['genre'], x))
    search_result = search.sort_values('result', ascending=False)['name'].head(5)
    for res in search_result.values:
        print('\t{}'.format(res))

Similar anime like Doron Coron:
	Ocha-ken: Ryokutto Monogatari
	Peng You Town
	Nisshin Seifun Group CM
	Noobow
	Histoire d&#039;Etty
Similar anime like Hengen Taima Yakou Karura Mau! Sendai Kokeshi Enka:
	Bishoujo Senshi Sailor Moon R: Make Up! Sailor Senshi
	Umezu Kazuo no Noroi
	Makura no Danshi
	Inuki Kanako Zekkyou Collection: Gakkou ga Kowai!
	C.L.A.Y.
Similar anime like Asylum Session:
	Dimension W: W no Tobira Online
	Dimension W Special
	Ouritsu Uchuugun: Honneamise no Tsubasa - Pilot Film
	Sakasama no Patema
	Mirai Arise
Similar anime like Gekitou! Crush Gear Turbo: Kaizabaan no Chousen:
	Judo Sanka
	Haikyuu!! OVA
	Chousoku Spinner
	Major 2nd
	Be Blues! Ao ni Nare
Similar anime like Monkey Punch: Manga Katsudou Daishashin - Crime Mate Special:
	Dengeki Oshioki Musume Gootaman R: Ai to Kanashimi no Final Battle
	Dengeki Oshioki Musume Gootaman: Gootaman Tanjou-hen
	Eguchi Hisashi no Kotobuki Gorou Show
	Haiyore! Nyaruko-san W OVA
	Monkey Punch no Sekai: Alice
Similar anime like

	Doukyuusei (OVA): Climax
	Ikenai Koto The Animation
	Boin
	Injuu Nerawareta Hanayome
	Furyou ni Hamerarete Jusei Suru Kyonyuu Okaasan The Animation
Similar anime like Be-Boy Kidnapp&#039;n Idol:
	Osakana wa Ami no Naka
	Hidari no O&#039;Clock!!
	Tsuru ni Notte: Tomoko no Bouken
	Kara no Kyoukai: Mirai Fukuin - Manner Movie
	Shitakiri Suzume
Similar anime like Kochinpa! Dainiki:
	Omake
	Cofun Gal no Coffy Campus Life
	Prince of Tennis: Mini Theater
	Mai-HiME: Kuro no Mai/Saigo no Bansan
	Curly
Similar anime like Shisha no Teikoku:
	Dasshutsu Gasshapon
	Gyakusatsu Kikan
	Harmony
	Sekai Kei Sekai Ron
	Shaka no Shougai
Similar anime like Moonlight Mile 2nd Season: Touch Down:
	Ginga Tetsudou 999 (ONA)
	Ginga Tetsudou 999
	Ginga Tetsudou 999: Akai Hoshi Betelgeuze - Inochi no Kagayaki
	Moonlight Mile 1st Season: Lift Off
	Ginga Tetsudou 999 for Planetarium
Similar anime like Tottoko Hamtarou Movie 4: Hamtaro to Fushigi no Oni no Emon Tou:
	Tottoko Hamtarou OVA 3: Hamuchanzu to Niji no Kuni

	InuYasha: Tenka Hadou no Ken
	InuYasha: Guren no Houraijima
	Saiunkoku Monogatari
	Saiunkoku Monogatari Soushuuhen
	InuYasha: Kagami no Naka no Mugenjo
Similar anime like Nine 2: Koibito Sengen:
	Nine: Original Han
	Ikkyuu-san (1978)
	Nine 3: Kanketsu-hen
	Apache Yakyuugun
	Golden Kids
Similar anime like Turning Girls:
	Yutori-chan
	Momoya x Peeping Life: Go en Desu yo! II
	Mainichi Kaasan
	Atashin&#039;chi
	Puchimas!!: Petit Petit iDOLM@STER
Similar anime like Wakaba*Girl: Onsen Tsukaritai:
	Yutori-chan
	Momoya x Peeping Life: Go en Desu yo! II
	Mainichi Kaasan
	Atashin&#039;chi
	Puchimas!!: Petit Petit iDOLM@STER
Similar anime like Hanamaru Youchien: Panda Neko Taisou:
	Saboten ga Nikui
	Minarai Chameleon
	Aruite Mikka!
	That&#039;s Hanakappa Musical: Pan to Gohan, Docchi nano!?
	Warenai Tamago
Similar anime like Two Down Full Base:
	Shin Pro Golfer Saru
	Pro Golfer Saru (1985)
	Eddie the Fast Break
	Pro Golfer Saru
	Pro Golfer Saru: Super Golf World e no Chousen!!
Similar anime lik

	Ojarumaru: Wasureta Mori no Hinata
	Ojarumaru: Mangetsu Road Kiki Ippatsu - Tama ni wa Maro mo Daibouken
	Shimajirou to Kujira no Uta
	Muumin (1972)
	Pokemon: Utae Meloetta - Rinka no Mi wo Sagase!
Similar anime like Andromeda Stories:
	Utawarerumono
	Saredo Tsumibito wa Ryuu to Odoru
	Captain Harlock
	Ginga Tetsudou 999: Eternal Fantasy
	Final Fantasy VII: Advent Children - Venice Film Festival Footage
Similar anime like Onegaiâ˜†Twins: Natsu wa Owaranai:
	Eromanga-sensei
	Maison Ikkoku: Prelude - Meguru Haru no Sakura no you ni...
	Futakoi Alternative
	Hanbun no Tsuki ga Noboru Sora
	Maison Ikkoku: Utsuriyuku Kisetsu no Naka de
Similar anime like Ore no Imouto ga Konnani Kawaii Wake ga Nai:
	Yurumates
	Oishinbo
	Tetsuko no Tabi
	Kariage-kun
	Ore no Imouto ga Konnani Kawaii Wake ga Nai.
Similar anime like Seikai no Senki II:
	Seikai no Senki III
	Seikai no Senki
	Seikai no Senki Special
	Seikai no Monshou
	Seikai no Monshou Special
Similar anime like Aoharu x Kikanjuu:
	Kinnikuman: G

	Future GPX Cyber Formula 11
	Future GPX Cyber Formula Saga
	Future GPX Cyber Formula Sin
	Future GPX Cyber Formula: Early Days Renewal
	Inazuma Eleven Go: Kyuukyoku no Kizuna Gryphon
Similar anime like Garden:
	Doukyuusei (OVA): Climax
	Ikenai Koto The Animation
	Boin
	Injuu Nerawareta Hanayome
	Furyou ni Hamerarete Jusei Suru Kyonyuu Okaasan The Animation
Similar anime like Wake up!! Tamala:
	Worku
	Tabi
	Arata Kangatari Picture Drama
	Dorami &amp; Doraemons: Robot School&#039;s Seven Mysteries
	Bishoujo Mobage: Mobami-chan
Similar anime like Kami nomi zo Shiru Sekai: Megami-hen:
	Kami nomi zo Shiru Sekai: Tenri-hen
	Kami nomi zo Shiru Sekai II
	Kami nomi zo Shiru Sekai
	Gugure! Kokkuri-san Specials
	Inu x Boku SS
Similar anime like Kotencotenco:
	Hey Yo Yorang
	Grimm Masterpiece Theater II
	Shimajirou to Fufu no Daibouken Movie: Sukue! Nana-iro no Hana
	Shimajirou Hesoka
	Shimajirou no Wow!
Similar anime like Kitte no Nai Okurimono:
	Minihams no Kekkon Song
	Watashi wa Tofu Desu
	Ha

...

	Flashback Game
	Swing Out Sisters (2014)
	Sei Brunehilde Gakuen Shoujo Kishidan to Junpaku no Panty The Animation
	Tsundere Inran Shoujo Sukumi
	Furueru Kuchibiru
Similar anime like Chocchan Monogatari:
	Ai Shoujo Pollyanna Story
	Maya no Isshou
	Shashinkan
	Alps no Shoujo Heidi: Heidi to Clara-hen
	Ihatov Gensou: Kenji no Haru
Similar anime like Ginga Kikoutai Majestic Prince Movie: Kakusei no Idenshi:
	Ginga Kikoutai Majestic Prince: Mirai e no Tsubasa
	Ginga Kikoutai Majestic Prince
	Sidonia no Kishi Movie
	Hakaima Sadamitsu
	Sidonia no Kishi: Daikyuu Wakusei Seneki
Similar anime like Hana to Mogura:
	Omake
	Cofun Gal no Coffy Campus Life
	Prince of Tennis: Mini Theater
	Mai-HiME: Kuro no Mai/Saigo no Bansan
	Curly
Similar anime like Lupin III: Episode 0 &quot;First Contact&quot;:
	Lupin III vs. Detective Conan
	Lupin III: Honoo no Kioku - Tokyo Crisis
	Lupin III: Walther P-38
	Lupin III: Lupin vs. Fukusei-ningen
	Lupin III: Hemingway Paper no Nazo
Similar anime like Gunsmith Cats:

	Flashback Game
	Swing Out Sisters (2014)
	Sei Brunehilde Gakuen Shoujo Kishidan to Junpaku no Panty The Animation
	Tsundere Inran Shoujo Sukumi
	Furueru Kuchibiru
Similar anime like Love Hina:
	Ane Log: Moyako Neesan no Honpen wo Tobidashite Tomaranai Monologue
	Ichigo 100%
	Saenai Heroine no Sodatekata: Ai to Seishun no Service Kai
	Oniichan no Koto nanka Zenzen Suki ja Nai n da kara ne!!
	Hanaukyou Maid-tai: La Verite
Similar anime like Kannazuki no Miko:
	Kyoushirou to Towa no Sora
	Kyoushirou to Towa no Sora Specials
	Kuttsukiboshi
	Kanamewo
	Touka Gettan
Similar anime like Bloodivores:
	Dance in the Vampire Bund
	Trinity Blood
	Dance in the Vampire Bund Recap
	Kizumonogatari III: Reiketsu-hen
	Zhen Hun Jie
Similar anime like Tanken Driland: 1000-nen no Mahou:
	Superflat First Love
	Druaga no Tou: The Aegis of Uruk - Jil no Bouken
	Chiisana Teien
	Peter Pan no Bouken
	Alice in Dreamland
Similar anime like Seichou Rap:
	Renkyori Enai
	Chuunen Punk
	Wake Up! Dodo
	World Calling
	Umo

	Prison School
	Mujaki no Rakuen
	Iketeru Futari
	Prison School OVA
	B Gata H Kei
Similar anime like Galilei Donna:
	Queen Emeraldas
	Uchuu Shounen Soran (Movie)
	Last Exile: Ginyoku no Fam Recaps
	Rockman Hoshi ni Negai wo
	Mass Effect: Paragon Lost
Similar anime like Fire Emblem:
	Chain Chronicle: Haecceitas no Hikari Part 3
	Meoteoldosa
	Magi: The Labyrinth of Magic
	Densetsu no Yuusha no Densetsu: Iris Report
	Magi: Sinbad no Bouken
Similar anime like Onegaiâ˜†Twins:
	Kakyuusei 2: Hitomi no Naka no Shoujo-tachi
	Tsuki wa Higashi ni Hi wa Nishi ni: Operation Sanctuary
	Date A Live: Date to Date
	To Heart 2 OVA
	Copihan
Similar anime like Fullmetal Alchemist: Premium Collection:
	One Piece: Jango no Dance Carnival
	Nissan Serena x One Piece 3D: Mugiwara Chase - Sennyuu!! Sauzando Sanii-gou
	VS Knight Lamune &amp; 40 Fire
	Francesca
	Dragon Ball Specials
Similar anime like Tsuru no Ongaeshi:
	Alice Tantei Kyoku
	Usagi no Mofy
	Spo-chan Taiketsu: Youkai Daikessen
	Poppoya-san: Nonki Ki

	Tales of Symphonia The Animation: Sekai Tougou-hen
	Shadow Skill: Kurudaryuu Kousatsuhou no Himitsu
	Shadow Skill (1996)
	Tales of Symphonia The Animation: Sylvarant-hen
	Tales of Symphonia The Animation: Tethe&#039;alla-hen
Similar anime like Mermaid Forest:
	Mermaid Forest OVA
	Mermaid&#039;s Scar
	Mugen Shinshi: Bouken Katsugeki-hen
	Ginga Tetsudou no Yoru: Fantasy Railroad in the Stars
	Jiok
Similar anime like Sayonara Zetsubou Sensei Special:
	Nekketsu Tantei Jimusho
	Nekketsu Jinmen Inu: Life Is Movie
	Mutsugo to Ouma no Monogatari
	Shakugan no Shana III (Final) Specials
	Reporter Blues
Similar anime like Magicalâ˜…Taruruuto-kun: Moeru! Yuujou no Mahou Taisen:
	Magicalâ˜…Taruruuto-kun Movie
	Magicalâ˜…Taruruuto-kun
	Magicalâ˜…Taruruuto-kun: Sukisuki Takoyaki!
	Little Witch Academia (TV)
	Majuu Senshi Luna Varga
Similar anime like Shuten Douji:
	Devilman: Yochou Sirene-hen
	Devilman: Tanjou-hen
	Tenpou Ibun: Ayakashi Ayashi
	Teito Monogatari
	Oni
Similar anime like Auction: Bijo 

	77Danui Bimil
	WeiÃŸ Kreuz OVA
	Sora no Arawashi
	Kentauros no Densetsu
	Tokyo Tribe 2
Similar anime like Mahou Shoujo Pretty Sammy:
	Nurse Witch Komugi-chan Magikarte
	Code Geass: Nunnally in Wonderland
	Zettai Bouei Leviathan
	Onegai My Melody
	Mahou Gakuen Lunar! Aoi Ryuu no Himitsu
Similar anime like Captain Tsubasa: Road to 2002:
	Meimon! Daisan Yakyuubu
	Captain Tsubasa: Sekai Daikessen!! Jr. World Cup
	Grappler Baki
	Ring ni Kakero 1
	Major Movie: Yuujou no Winning Shot
Similar anime like Doraemon Movie 21: Nobita no Taiyou Ou Densetsu:
	Doraemon: Ganbare! Gian!!
	Doraemon Movie 23: Nobita to Robot Kingdom
	Doraemon Movie 12: Nobita no Dorabian Nights
	Doraemon Movie 28: Nobita to Midori no Kyojin Den
	Doraemon Movie 10: Nobita no Nippon Tanjou
Similar anime like Kanokon Specials:
	Senran Kagura Specials
	Hyakka Ryouran: Samurai Girls Specials
	Hyakka Ryouran: Samurai After Specials
	Grisaia no Meikyuu Special
	Lemon Angel (1988/II)
Similar anime like Oyone to Mattsan: Ai no Ut